### Deep Sentiment Analysis Tutorial

### Setup GPU
- On Google Colab make sure you select Python 3/GPU runtime before running the code

In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


### Download Data

In [2]:
# !mkdir data/
# !wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -P data/

In [3]:
# %%time
# !tar -xzf data/aclImdb_v1.tar.gz -C data/

### Imports

In [4]:
import os
import re
import nltk
from collections import Counter
from tqdm import tqdm_notebook
import numpy as np
import tensorflow as tf
from tensorflow.contrib import seq2seq
from tensorflow.contrib.rnn import DropoutWrapper
import random

In [60]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/bishal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
MAX_SEQ_LEN = 50
BATCH_SIZE = 64

In [7]:
class Lang:
    def __init__(self, counter, vocab_size):
        self.word2id = {}
        self.id2word = {}
        self.pad = "<PAD>"
        self.sos = "<SOS>"
        self.eos = "<EOS>"
        self.unk = "<UNK>"
        
        self.ipad = 0
        self.isos = 1
        self.ieos = 2
        self.iunk = 3
        
        self.word2id[self.pad] = 0
        self.word2id[self.sos] = 1
        self.word2id[self.eos] = 2
        self.word2id[self.unk] = 3
        
        self.id2word[0] = self.pad
        self.id2word[1] = self.sos
        self.id2word[2] = self.eos
        self.id2word[3] = self.unk
        
        curr_id = 4
        for w, c in counter.most_common(vocab_size):
            self.word2id[w] = curr_id
            self.id2word[curr_id] = w
            curr_id += 1
    
    def encodeSentence(self, wseq, max_len=-1):
        # wseq = nltk.tokenize.word_tokenize(s.lower().strip())
        if max_len == -1:
            return [self.word2id[w] if w in self.word2id else self.iunk for w in wseq]
        else:
            return ([self.word2id[w] if w in self.word2id else self.iunk for w in wseq] + [self.ieos] + [self.ipad]*max_len)[:max_len]
        
    def encodeSentence2(self, wseq, max_len=-1):
        # wseq = nltk.tokenize.word_tokenize(s.lower().strip()) 
        return min(max_len, len(wseq)+1), \
            ([self.word2id[w] if w in self.word2id else self.iunk for w in wseq] + \
                [self.ieos] + [self.ipad]*max_len)[:max_len]
    
    def decodeSentence(self, id_seq):
        id_seq = np.array(id_seq + [self.ieos])
        j = np.argmax(id_seq==self.ieos)
        s = ' '.join([self.id2word[x] for x in id_seq[:j]])
        s = s.replace(self.unk, "UNK")
        return s

### Let's read in the data

In [8]:
data_folder = 'data/aclImdb/'

In [9]:
rp = os.path.join(data_folder, 'train/pos')
train_positive = [os.path.join(rp, f) for f in os.listdir(rp)]
rp = os.path.join(data_folder, 'train/neg')
train_negative = [os.path.join(rp, f) for f in os.listdir(rp)]

rp = os.path.join(data_folder, 'test/pos')
test_positive = [os.path.join(rp, f) for f in os.listdir(rp)]
rp = os.path.join(data_folder, 'test/neg')
test_negative = [os.path.join(rp, f) for f in os.listdir(rp)]

#### Limit number of samples
To quickly train a small model, consider setting n_train and n_test to some relatively small numbers e.g. `1000`. Set, 
`n_train = n_test = -1` to use all the samples available.

In [10]:
n_train = 2500
n_test = 2500

In [11]:
re_html_cleaner = re.compile(r"<.*?>")

In [12]:
en_counter = Counter()
train_data = []
for _fname in tqdm_notebook(train_positive[:n_train], desc="Crunching +ve samples: "):
    with open(_fname) as f:
        text = f.read().strip()
        text = re_html_cleaner.sub(" ", text)
        wseq = nltk.tokenize.word_tokenize(text.lower())
        en_counter += Counter(wseq)
        train_data.append((wseq, 1))
        
for _fname in tqdm_notebook(train_negative[:n_train], desc="Crunching -ve samples: "):
    with open(_fname) as f:
        text = f.read().strip()
        text = re_html_cleaner.sub(" ", text)
        wseq = nltk.tokenize.word_tokenize(text.lower())
        en_counter += Counter(wseq)
        train_data.append((wseq, 0))

In [13]:
test_data = []
for _fname in tqdm_notebook(test_positive[:n_test], desc="Crunching +ve samples: "):
    with open(_fname) as f:
        text = f.read().strip()
        text = re_html_cleaner.sub(" ", text)
        wseq = nltk.tokenize.word_tokenize(text.lower())
        test_data.append((wseq, 1))
        
for _fname in tqdm_notebook(test_negative[:n_test], desc="Crunching -ve samples: "):
    with open(_fname) as f:
        text = f.read().strip()
        text = re_html_cleaner.sub(" ", text)
        wseq = nltk.tokenize.word_tokenize(text.lower())
        test_data.append((wseq, 0))

In [14]:
# A few sample english words
print("\nMost common en words in dataset:\n", en_counter.most_common(10))

print("\nTotal (en)words gathered from dataset:", len(en_counter))


Most common en words in dataset:
 [('the', 66942), (',', 55132), ('.', 54084), ('and', 32778), ('a', 32376), ('of', 29094), ('to', 27116), ('is', 22131), ('it', 19161), ('in', 18658)]

Total (en)words gathered from dataset: 46829


In [15]:
V = 10000

In [16]:
en_lang = Lang(en_counter, V)

In [17]:
wseq = nltk.tokenize.word_tokenize("Where are you going?".lower())
print("Test en encoding:", en_lang.encodeSentence(wseq))
print("Test en decoding:", en_lang.decodeSentence(en_lang.encodeSentence(wseq, 10)))

Test en encoding: [127, 32, 28, 182, 59]
Test en decoding: where are you going ?


### The RNN based Sentence Classifier architecture
- We will implement a RNN based classifier architecture for sentiment analysis in Tensorflow r1.13.1 / r1.14
- Debugging Tip: Always keep track of tensor dimensions!
- **Tensorflow Computation Graph** - We will build a tf computation graph first. This is the representation used by tf for any neural network architecture. Once the computation graph is built, you can feed data to it for training or inference

#### Word Embedding Matrix

In [18]:
en_word_emb_matrix = tf.get_variable("en_word_emb_matrix", (V, 300), dtype=tf.float32)

Instructions for updating:
Colocations handled automatically by placer.


#### Placeholders

In [19]:
keep_prob = tf.placeholder(tf.float32)

In [20]:
input_ids = tf.placeholder(tf.int32, (None, MAX_SEQ_LEN))
input_lens = tf.placeholder(tf.int32, (None, ))

In [21]:
y_placeholder = tf.placeholder(tf.int32, (None,))

#### Tensorflow Graphs

In [22]:
input_emb = tf.nn.embedding_lookup(en_word_emb_matrix, input_ids)

In [23]:
input_emb.shape

TensorShape([Dimension(None), Dimension(50), Dimension(300)])

#### Encoder

##### RNN Units

In [24]:
encoder_cell = tf.nn.rnn_cell.GRUCell(128)
encoder_cell = DropoutWrapper(encoder_cell, output_keep_prob=keep_prob)

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.


In [25]:
enc_outputs, enc_state = tf.nn.dynamic_rnn(
    encoder_cell, input_emb, sequence_length=input_lens, initial_state=encoder_cell.zero_state(BATCH_SIZE, dtype=tf.float32)
)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [26]:
enc_outputs.shape

TensorShape([Dimension(64), Dimension(50), Dimension(128)])

In [27]:
enc_state.shape

TensorShape([Dimension(64), Dimension(128)])

### Classifier Layer

In [28]:
dense_layer = tf.layers.Dense(1)

#### Approaches:
As input to the final linear layers use mean of the hidden states?

or

As input to the final linear layers use the last hidden state?

##### Approch 1: Take mean of enc_outputs across dimension 1
- **IMPORTANT:** Need to **mask** the positions in input sentence that doesn't contain any inputs

In [45]:
masks = tf.sequence_mask(input_lens, MAX_SEQ_LEN, dtype=tf.float32, name='masks')
class_prob = tf.nn.sigmoid(dense_layer(tf.reduce_mean(enc_outputs*masks[:, :, None], 1))) 

In [46]:
class_prob.shape

TensorShape([Dimension(64), Dimension(1)])

##### Approch 2: Use enc_state (final hidden state)

In [31]:
# class_prob = dense_layer(enc_state) 
# class_prob.shape

#### Loss and Optimizers [softmax_cross_entropy]
Note that `onehot_labels` and `logits` must have the same shape, e.g. `[batch_size, num_classes]`

In [47]:
print(y_placeholder.shape)
print(class_prob.shape)

(?,)
(64, 1)


In [75]:
# Loss function - softmax cross entropy
y_ = tf.cast(y_placeholder[:, None], dtype=tf.float32)
cost = -y_*tf.log(class_prob + 1e-12) - (1-y_)*tf.log(1-class_prob + 1e-12)
cost = tf.reduce_mean(cost)
# cost = tf.losses.softmax_cross_entropy(
#     onehot_labels=tf.stack([1-y_placeholder, y_placeholder], 1), # Conversion to one-hot
#     logits=tf.concat([1-class_prob, class_prob], -1)
# )
# cost = tf.losses.sigmoid_cross_entropy(
#     y_placeholder[:, None], # Conversion to one-hot
#     class_prob
# )

# Optimizer
optimizer = tf.train.AdamOptimizer(0.0001)

In [76]:
train_op = optimizer.minimize(cost)

In [77]:
init = tf.global_variables_initializer()

#### Tensorflow Sessions

In [78]:
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True

In [79]:
sess = tf.InteractiveSession(config=sess_config)
sess.run(init)

/home/bishal/miniconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


#### Minibatch Training

In [80]:
random.seed(41)

In [81]:
random.shuffle(train_data)

In [82]:
train_n = len(train_data)

In [83]:
test_n = len(test_data)

In [86]:
def small_test():
    all_true = []
    all_preds = []
    for m in range(0, test_n, BATCH_SIZE):
        n = m + BATCH_SIZE
        if n > test_n:
            break

        input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
        input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
        true_class_batch = np.zeros((BATCH_SIZE))
        for i in range(m, n):
            b,a = en_lang.encodeSentence2(train_data[i][0], MAX_SEQ_LEN)
            input_batch[i-m,:] = a
            input_lens_batch[i-m] = b
            true_class_batch[i-m] = train_data[i][1]

        feed_dict={
            input_ids: input_batch,
            input_lens: input_lens_batch,
            keep_prob: 1.0
        }
        pred_batch = sess.run(class_prob, feed_dict=feed_dict)
        # acc = accuracy_score(true_class_batch, pred_batch > 0.5)
        all_true.extend(list(true_class_batch))
        all_preds.extend(list(pred_batch[:,0]))
    
    all_true = np.array(all_true)
    all_preds = np.array(all_preds)
    prec = precision_score(all_true, all_preds > 0.5)*100
    rec = recall_score(all_true, all_preds > 0.5)*100
    f1 = f1_score(all_true, all_preds > 0.5)*100
    print(f"Precision: {prec:2.2F}, Recall: {rec:2.2F}, F1-Score: {f1:2.2F}")

In [ ]:
for _e in range(10):
    # Mix things up a bit.
    random.shuffle(train_data)
    pbar = tqdm_notebook(range(0, train_n, BATCH_SIZE))
    batch_loss = 0
    bxi = 0
    for m in pbar:
        n = m + BATCH_SIZE
        if n <= train_n:
            # print("Epoch Complete... \n")

            input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
            input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
            true_class_batch = np.zeros((BATCH_SIZE))
            for i in range(m, n):
                b,a = en_lang.encodeSentence2(train_data[i][0], MAX_SEQ_LEN)
                input_batch[i-m,:] = a
                input_lens_batch[i-m] = b
                true_class_batch[i-m] = train_data[i][1]

            feed_dict={
                input_ids: input_batch,
                input_lens: input_lens_batch,
                y_placeholder: true_class_batch,
                keep_prob: 0.8 
            }
            sess.run(train_op, feed_dict=feed_dict)
            batch_loss += sess.run(cost, feed_dict=feed_dict)
            pbar.set_description(f"Epoch: {_e} >> Loss: {batch_loss/(bxi+1):2.2F}:")
            bxi += 1
            if (1 + n//BATCH_SIZE) % 10 == 0:
                small_test()

Precision: 82.05, Recall: 5.13, F1-Score: 9.65
Precision: 91.18, Recall: 1.24, F1-Score: 2.45
Precision: 90.76, Recall: 4.33, F1-Score: 8.26
Precision: 88.27, Recall: 13.26, F1-Score: 23.06
Precision: 86.42, Recall: 23.72, F1-Score: 37.22
Precision: 82.16, Recall: 39.30, F1-Score: 53.17
Precision: 73.32, Recall: 62.22, F1-Score: 67.32


Precision: 67.75, Recall: 78.33, F1-Score: 72.66
Precision: 74.86, Recall: 59.99, F1-Score: 66.61
Precision: 70.30, Recall: 77.93, F1-Score: 73.92
Precision: 75.98, Recall: 77.89, F1-Score: 76.92
Precision: 80.80, Recall: 72.13, F1-Score: 76.22
Precision: 77.21, Recall: 83.58, F1-Score: 80.27
Precision: 77.70, Recall: 86.10, F1-Score: 81.69


Precision: 78.65, Recall: 89.14, F1-Score: 83.57
Precision: 85.99, Recall: 79.41, F1-Score: 82.57
Precision: 84.01, Recall: 86.30, F1-Score: 85.14
Precision: 85.00, Recall: 88.30, F1-Score: 86.62
Precision: 87.20, Recall: 88.42, F1-Score: 87.81
Precision: 89.17, Recall: 88.38, F1-Score: 88.77
Precision: 88.02, Recall: 91.51, F1-Score: 89.73


Precision: 89.73, Recall: 91.67, F1-Score: 90.69
Precision: 89.65, Recall: 91.91, F1-Score: 90.77
Precision: 93.52, Recall: 86.14, F1-Score: 89.68
Precision: 86.85, Recall: 95.47, F1-Score: 90.96
Precision: 92.92, Recall: 91.39, F1-Score: 92.15
Precision: 91.73, Recall: 94.23, F1-Score: 92.97
Precision: 93.37, Recall: 93.63, F1-Score: 93.50


Precision: 94.41, Recall: 93.47, F1-Score: 93.94
Precision: 93.48, Recall: 95.39, F1-Score: 94.43
Precision: 96.21, Recall: 90.46, F1-Score: 93.25
Precision: 91.84, Recall: 97.43, F1-Score: 94.55
Precision: 95.34, Recall: 95.11, F1-Score: 95.22
Precision: 93.65, Recall: 96.91, F1-Score: 95.25
Precision: 97.04, Recall: 94.47, F1-Score: 95.74


Precision: 96.20, Recall: 96.31, F1-Score: 96.25
